# Data Extraction Pipeline - Big 5 European Leagues

**Phase 1: Data Foundation**

This notebook extracts match history and advanced statistics for the Big 5 European football leagues using the `soccerdata` library. Each league is processed in its own isolated cell, allowing independent execution, debugging, and data saving.

### Data Sources
| Source | Method | Description |
|--------|--------|-------------|
| **MatchHistory** | `read_games()` | Historic match results & betting odds from football-data.co.uk |
| **Understat** | `read_schedule()` | Match schedule with xG / advanced stats from understat.com |
| **ClubElo** | `read_team_history()` | Full Elo rating history per team from clubelo.com |

### Output
All CSVs are saved to `data/raw/` with a strict naming convention:
- `{League_Code}_MatchHistory.csv`
- `{League_Code}_Understat.csv`
- `ClubElo_Master.csv`

In [1]:
# =============================================================================
# Cell 1: Setup & Initialization
# =============================================================================

try:
    import soccerdata as sd
except ImportError:
    %pip install soccerdata
    import soccerdata as sd

import pandas as pd
import os
import time
import warnings
import glob

warnings.filterwarnings('ignore')

# --- Create output directory ---
output_dir = os.path.join('data', 'raw')
os.makedirs(output_dir, exist_ok=True)
print(f'Output directory ready: {os.path.abspath(output_dir)}')

# --- Define seasons (14/15 through 24/25 = 11 complete seasons) ---
seasons = [str(i) for i in range(2014, 2025)]
print(f'Seasons configured: {seasons[0]}/{int(seasons[0])+1} through {seasons[-1]}/{int(seasons[-1])+1}')

# --- League configuration ---
leagues = {
    'ENG-Premier League': 'ENG_Premier_League',
    'ESP-La Liga':        'ESP_La_Liga',
    'GER-Bundesliga':     'GER_Bundesliga',
    'ITA-Serie A':        'ITA_Serie_A',
    'FRA-Ligue 1':        'FRA_Ligue_1',
}

def extract_league_data(league_id, league_code, seasons):
    """
    Extract MatchHistory and Understat data for a single league.
    Handles errors per-season so partial data is always saved.
    """
    mh_path = os.path.join(output_dir, f'{league_code}_MatchHistory.csv')
    us_path = os.path.join(output_dir, f'{league_code}_Understat.csv')
    
    # --- MatchHistory ---
    print(f'  Fetching MatchHistory for {league_id}...')
    mh_frames = []
    for season in seasons:
        try:
            mh = sd.MatchHistory(leagues=league_id, seasons=season)
            df = mh.read_games()
            if df is not None and len(df) > 0:
                mh_frames.append(df)
        except Exception as e:
            print(f'    [WARN] MatchHistory failed for {league_id} season {season}: {e}')
    
    if mh_frames:
        mh_all = pd.concat(mh_frames, ignore_index=False)
        mh_all.to_csv(mh_path)
        print(f'    [OK] Saved {len(mh_all)} rows -> {mh_path}')
    else:
        print(f'    [FAIL] No MatchHistory data retrieved for {league_id}')
    
    # --- Understat ---
    print(f'  Fetching Understat schedule for {league_id}...')
    us_frames = []
    for season in seasons:
        try:
            us = sd.Understat(leagues=league_id, seasons=season)
            df = us.read_schedule()
            if df is not None and len(df) > 0:
                us_frames.append(df)
        except Exception as e:
            print(f'    [WARN] Understat failed for {league_id} season {season}: {e}')
    
    if us_frames:
        us_all = pd.concat(us_frames, ignore_index=False)
        us_all.to_csv(us_path)
        print(f'    [OK] Saved {len(us_all)} rows -> {us_path}')
    else:
        print(f'    [FAIL] No Understat data retrieved for {league_id}')

print('\nSetup complete. Run the cells below in order.')

[02/23/26 22:52:46] INFO     No custom team name replacements found. You can configure these in       ]8;id=832717;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=128539;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py#92\92]8;;\
                             C:\Users\ljega\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=123376;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=796876;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_config.py#198\198]8;;\
                             C:\Users\ljega\soccerdata\config\league_dict.json.                                    

Output directory ready: c:\Users\ljega\Downloads\MSBA\Sports Analytics Project\notebooks\data\raw
Seasons configured: 2014/2015 through 2024/2025

Setup complete. Run the cells below in order.


In [2]:
# =============================================================================
# Cell 1.5: Pre-download MatchHistory CSVs (bypass bot detection)
# =============================================================================
# football-data.co.uk blocks automated requests with 503 errors.
# This cell downloads CSVs using browser-like headers and saves them
# to soccerdata's cache so the library uses cached files instead.

import requests
from pathlib import Path

cache_dir = Path.home() / 'soccerdata' / 'data' / 'MatchHistory'
cache_dir.mkdir(parents=True, exist_ok=True)

browser_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.football-data.co.uk/',
}

mh_league_codes = {
    'ENG-Premier League': 'E0',
    'ESP-La Liga':        'SP1',
    'GER-Bundesliga':     'D1',
    'ITA-Serie A':        'I1',
    'FRA-Ligue 1':        'F1',
}

def season_code(year):
    return f'{int(year) % 100:02d}{(int(year)+1) % 100:02d}'

base_url = 'https://www.football-data.co.uk/mmz4281'

downloaded = 0
skipped = 0
failed = 0

for league_name, code in mh_league_codes.items():
    print(f'\nPre-downloading MatchHistory for {league_name} ({code})...')
    for season in seasons:
        sc = season_code(season)
        filename = f'{code}_{sc}.csv'
        filepath = cache_dir / filename
        
        if filepath.exists() and filepath.stat().st_size > 100:
            skipped += 1
            continue
        
        url = f'{base_url}/{sc}/{code}.csv'
        try:
            resp = requests.get(url, headers=browser_headers, timeout=30)
            resp.raise_for_status()
            filepath.write_bytes(resp.content)
            downloaded += 1
            print(f'    [OK] {filename} ({len(resp.content):,} bytes)')
            time.sleep(1)
        except Exception as e:
            failed += 1
            print(f'    [WARN] {filename}: {e}')

print(f'\nSummary: {downloaded} downloaded, {skipped} already cached, {failed} failed')
print(f'Cache location: {cache_dir}')
print('\nMatchHistory cache is ready. Run the league cells below.')


Pre-downloading MatchHistory for ENG-Premier League (E0)...

Pre-downloading MatchHistory for ESP-La Liga (SP1)...

Pre-downloading MatchHistory for GER-Bundesliga (D1)...

Pre-downloading MatchHistory for ITA-Serie A (I1)...

Pre-downloading MatchHistory for FRA-Ligue 1 (F1)...

Summary: 0 downloaded, 55 already cached, 0 failed
Cache location: C:\Users\ljega\soccerdata\data\MatchHistory

MatchHistory cache is ready. Run the league cells below.


In [3]:
# =============================================================================
# Cell 2: Premier League (ENG-Premier League)
# =============================================================================
time.sleep(2)  # Rate-limit guard

print('Extracting Premier League data...')
print('=' * 50)

extract_league_data(
    league_id='ENG-Premier League',
    league_code='ENG_Premier_League',
    seasons=seasons
)

print('\nPremier League extraction complete.')

Extracting Premier League data...
  Fetching MatchHistory for ENG-Premier League...


[02/23/26 22:54:06] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=290789;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=520195;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[2026-02-23 22:54:06] INFO     TLSLibrary:_load_library:397 - Successfully loaded TLS library: C:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\tls_requests\bin\tls-client-xgo-1.13.1-windows-amd64.dll


                    INFO     Successfully loaded TLS library:                                      ]8;id=158918;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\tls_requests\models\libraries.py\libraries.py]8;;\:]8;id=43224;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\tls_requests\models\libraries.py#397\397]8;;\
                             C:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packa                 
                             ges\tls_requests\bin\tls-client-xgo-1.13.1-windows-amd64.dll                          

                    INFO     Parsing league=E0 season=1415                                      ]8;id=413541;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=537001;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

...

                    INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=891924;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=921730;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:54:14] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=265757;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=209056;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:54:15] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=222840;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=282779;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:54:16] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=635600;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=449428;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

    [OK] Saved 4180 rows -> data\raw\ENG_Premier_League_Understat.csv

Premier League extraction complete.


In [4]:
# =============================================================================
# Cell 3: La Liga (ESP-La Liga)
# =============================================================================
time.sleep(2)  # Rate-limit guard

print('Extracting La Liga data...')
print('=' * 50)

extract_league_data(
    league_id='ESP-La Liga',
    league_code='ESP_La_Liga',
    seasons=seasons
)

print('\nLa Liga extraction complete.')

Extracting La Liga data...
  Fetching MatchHistory for ESP-La Liga...


[02/23/26 22:54:29] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=568113;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=399447;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Parsing league=SP1 season=1415                                     ]8;id=684701;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=329840;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

                    INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=952092;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=341667;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Parsing league=SP1 season=1516                                     ]8;id=549684;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=613618;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

...

[02/23/26 22:54:36] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=805395;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=416674;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:54:37] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=221123;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=374563;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=338102;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=144266;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:54:38] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=730966;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=126113;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

    [OK] Saved 4180 rows -> data\raw\ESP_La_Liga_Understat.csv

La Liga extraction complete.


In [5]:
# =============================================================================
# Cell 4: Bundesliga (GER-Bundesliga)
# =============================================================================
time.sleep(2)  # Rate-limit guard

print('Extracting Bundesliga data...')
print('=' * 50)

extract_league_data(
    league_id='GER-Bundesliga',
    league_code='GER_Bundesliga',
    seasons=seasons
)

print('\nBundesliga extraction complete.')

Extracting Bundesliga data...
  Fetching MatchHistory for GER-Bundesliga...


[02/23/26 22:54:59] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=24910;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=263534;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Parsing league=D1 season=1415                                      ]8;id=18885;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=409906;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

                    INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=716411;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=529081;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Parsing league=D1 season=1516                                      ]8;id=105699;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=668205;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

...

[02/23/26 22:55:06] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=790071;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=355244;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:07] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=618188;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=483820;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:08] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=598081;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=149475;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:09] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=198500;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=364326;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

    [OK] Saved 3366 rows -> data\raw\GER_Bundesliga_Understat.csv

Bundesliga extraction complete.


In [6]:
# =============================================================================
# Cell 5: Serie A (ITA-Serie A)
# =============================================================================
time.sleep(2)  # Rate-limit guard

print('Extracting Serie A data...')
print('=' * 50)

extract_league_data(
    league_id='ITA-Serie A',
    league_code='ITA_Serie_A',
    seasons=seasons
)

print('\nSerie A extraction complete.')

Extracting Serie A data...
  Fetching MatchHistory for ITA-Serie A...


[02/23/26 22:55:21] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=660151;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=92759;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Parsing league=I1 season=1415                                      ]8;id=970648;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=197289;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

                    INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=586944;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=364936;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Parsing league=I1 season=1516                                      ]8;id=348251;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=649690;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

...

[02/23/26 22:55:28] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=230037;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=886753;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=911406;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=234016;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:29] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=287364;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=697312;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:30] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=405934;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=792127;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

    [OK] Saved 4180 rows -> data\raw\ITA_Serie_A_Understat.csv

Serie A extraction complete.


In [7]:
# =============================================================================
# Cell 6: Ligue 1 (FRA-Ligue 1)
# =============================================================================
time.sleep(2)  # Rate-limit guard

print('Extracting Ligue 1 data...')
print('=' * 50)

extract_league_data(
    league_id='FRA-Ligue 1',
    league_code='FRA_Ligue_1',
    seasons=seasons
)

print('\nLigue 1 extraction complete.')

Extracting Ligue 1 data...
  Fetching MatchHistory for FRA-Ligue 1...


[02/23/26 22:55:35] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=803939;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=422123;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Parsing league=F1 season=1415                                      ]8;id=491368;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=562525;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

                    INFO     Saving cached data to C:\Users\ljega\soccerdata\data\MatchHistory       ]8;id=182211;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=324570;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Parsing league=F1 season=1516                                      ]8;id=796670;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py\match_history.py]8;;\:]8;id=253009;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\match_history.py#17\17]8;;\

...

[02/23/26 22:55:42] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=394402;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=944940;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:43] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=653510;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=262458;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:44] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=508759;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=593494;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

                    INFO     Saving cached data to C:\Users\ljega\soccerdata\data\Understat          ]8;id=48235;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=853880;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

    [OK] Saved 4032 rows -> data\raw\FRA_Ligue_1_Understat.csv

Ligue 1 extraction complete.


In [8]:
# =============================================================================
# Cell 7: ClubElo Aggregation
# =============================================================================
print('Fetching ClubElo ratings for all Big 5 leagues...')
print('=' * 50)

elo_path = os.path.join(output_dir, 'ClubElo_Master.csv')

# --- Name mapping: football-data.co.uk -> ClubElo ---
# These teams use abbreviated names in MatchHistory that don't match ClubElo.
clubelo_name_map = {
    'Ajaccio GFCO':          'Ajaccio',
    'Ath Bilbao':            'Bilbao',
    'Ath Madrid':            'Atletico',
    'Bayern Munich':         'Bayern',
    'Ein Frankfurt':         'Frankfurt',
    'Espanol':               'Espanyol',
    'Evian Thonon Gaillard': 'Evian TG',
    'FC Koln':               'Koeln',
    'Fortuna Dusseldorf':    'Duesseldorf',
    'Holstein Kiel':         'Holstein',
    'La Coruna':             'Depor',
    "M'gladbach":            'Gladbach',
    "Nott'm Forest":         'Forest',
    'Nurnberg':              'Nuernberg',
    'Sp Gijon':              'Gijon',
    'St Etienne':            'Saint-Etienne',
    'Vallecano':             'Rayo Vallecano',
    'Werder Bremen':         'Werder',
}

# Collect all unique team names from saved MatchHistory files
mh_files = glob.glob(os.path.join(output_dir, '*_MatchHistory.csv'))
raw_teams = set()

for f in mh_files:
    try:
        df = pd.read_csv(f)
        home_col = [c for c in df.columns if 'home' in c.lower() and 'team' in c.lower()]
        away_col = [c for c in df.columns if 'away' in c.lower() and 'team' in c.lower()]
        if home_col:
            raw_teams.update(df[home_col[0]].dropna().unique())
        if away_col:
            raw_teams.update(df[away_col[0]].dropna().unique())
    except Exception as e:
        print(f'  [WARN] Could not read {f}: {e}')

# Apply name mapping
all_teams = {clubelo_name_map.get(t, t) for t in raw_teams}
mapped_count = sum(1 for t in raw_teams if t in clubelo_name_map)
print(f'  Found {len(raw_teams)} unique teams ({mapped_count} names remapped for ClubElo)')

# Fetch ClubElo data per team
elo_frames = []
failed_teams = []

for i, team in enumerate(sorted(all_teams)):
    try:
        elo = sd.ClubElo()
        df = elo.read_team_history(team)
        if df is not None and len(df) > 0:
            elo_frames.append(df)
            if (i + 1) % 20 == 0:
                print(f'  Processed {i+1}/{len(all_teams)} teams...')
        time.sleep(0.5)
    except Exception as e:
        failed_teams.append(team)

if elo_frames:
    elo_all = pd.concat(elo_frames, ignore_index=True)
    elo_all.to_csv(elo_path, index=False)
    print(f'\n  [OK] Saved {len(elo_all)} Elo records -> {elo_path}')
else:
    print('  [FAIL] No ClubElo data retrieved')

if failed_teams:
    print(f'  [WARN] {len(failed_teams)} teams could not be matched in ClubElo:')
    for t in sorted(failed_teams):
        print(f'      - {t}')
else:
    print('  All teams matched successfully.')

print('\nClubElo aggregation complete.')

Fetching ClubElo ratings for all Big 5 leagues...
  Found 163 unique teams (18 names remapped for ClubElo)


[02/23/26 22:55:55] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\ClubElo            ]8;id=42434;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=958673;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:57] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\ClubElo            ]8;id=646122;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=17487;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:55:59] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\ClubElo            ]8;id=726177;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=756985;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 22:56:02] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\ClubElo            ]8;id=138092;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=668944;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

...

[02/23/26 23:02:19] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\ClubElo            ]8;id=243718;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=922897;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

  Processed 160/162 teams...


[02/23/26 23:02:22] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\ClubElo            ]8;id=868657;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=705564;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\

[02/23/26 23:02:24] INFO     Saving cached data to C:\Users\ljega\soccerdata\data\ClubElo            ]8;id=904552;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=445620;file://c:\Users\ljega\AppData\Local\Programs\Python\Python312\Lib\site-packages\soccerdata\_common.py#249\249]8;;\


  [OK] Saved 699287 Elo records -> data\raw\ClubElo_Master.csv
  [WARN] 1 teams could not be matched in ClubElo:
      - Saint-Etienne

ClubElo aggregation complete.


In [9]:
# =============================================================================
# Cell 8: Verification Summary
# =============================================================================
print('DATA EXTRACTION VERIFICATION')
print('=' * 60)

all_files = glob.glob(os.path.join(output_dir, '*.csv'))
total_rows = 0

for f in sorted(all_files):
    try:
        df = pd.read_csv(f)
        size_kb = os.path.getsize(f) / 1024
        total_rows += len(df)
        status = '[OK]  ' if len(df) > 0 else '[FAIL]'
        print(f'  {status} {os.path.basename(f):45s} {len(df):>7,} rows  ({size_kb:>8,.1f} KB)')
    except Exception as e:
        print(f'  [FAIL] {os.path.basename(f):45s} ERROR: {e}')

print(f'\n{"=" * 60}')
print(f'  Total files: {len(all_files)}')
print(f'  Total rows:  {total_rows:,}')
print(f'  Location:    {os.path.abspath(output_dir)}')
print(f'\nData extraction pipeline complete.')

DATA EXTRACTION VERIFICATION
  [OK]   ClubElo_Master.csv                            699,287 rows  (29,744.5 KB)
  [OK]   ENG_Premier_League_MatchHistory.csv             4,180 rows  ( 2,220.9 KB)
  [OK]   ENG_Premier_League_Understat.csv                4,180 rows  (   791.0 KB)
  [OK]   ESP_La_Liga_MatchHistory.csv                    4,180 rows  ( 2,006.4 KB)
  [OK]   ESP_La_Liga_Understat.csv                       4,180 rows  (   748.4 KB)
  [OK]   FRA_Ligue_1_MatchHistory.csv                    3,931 rows  ( 1,974.7 KB)
  [OK]   FRA_Ligue_1_Understat.csv                       4,032 rows  (   690.1 KB)
  [OK]   GER_Bundesliga_MatchHistory.csv                 3,366 rows  ( 1,739.8 KB)
  [OK]   GER_Bundesliga_Understat.csv                    3,366 rows  (   656.5 KB)
  [OK]   ITA_Serie_A_MatchHistory.csv                    4,180 rows  ( 2,119.2 KB)
  [OK]   ITA_Serie_A_Understat.csv                       4,180 rows  (   707.0 KB)

  Total files: 11
  Total rows:  739,062
  Location:    c